In [ ]:
import pyvacon
import pyvacon.marketdata.testdata as mkt_testdata
import pyvacon.tools.enums as enums
import pyvacon.marketdata.plot as mkt_plot
import pyvacon.models.plot as model_plot
import pyvacon.models.tools as model_tools
import pyvacon.analytics as analytics
import pyvacon.tools.converter as converter

from matplotlib.lines import Line2D
from matplotlib.patches import Patch, Rectangle
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import matplotlib.transforms as mtransforms
%matplotlib inline

import datetime as dt
import math
import numpy as np

from scipy import stats
import plotly
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import ipywidgets as widgets

In [ ]:
# We define some constants, which we'll use repeatedly throughout this notebook.
color_main = 'tab:blue'
color_highlight = 'tab:orange'
grid_alpha = 0.4
daycounter_type_standard = enums.DayCounter.ACTACT
interpolation_type_standard = enums.InterpolationType.LINEAR
extrapolation_type_standard = enums.ExtrapolationType.NONE

# Interest Rate Shock Scenarios

In [ ]:
# define functions to help compute the shock scenarios

def shortRateShock(t, basepoints):
    if isinstance(t, (list, np.ndarray)):
        result = []
        for x in t:
            result.append(shortRateShock(x, basepoints))
        #print('list')
        return result
    #print('scalar')
    #print(type(t))
    return np.exp(-t/4) * basepoints


def longRateShock(t, basepoints):
    if isinstance(t, (list, np.ndarray)):
        result = []
        for x in t:
            result.append(longRateShock(x, basepoints))
        return result
    return (1-math.exp(-t/4)) * basepoints


def steepener(t, basepointsShort, basepointsLong):
    if isinstance(t, (list, np.ndarray)):
        result = []
        for x in t:
            result.append(steepener(x, basepointsShort, basepointsLong))
        return result
    return -0.65 * shortRateShock(t, basepointsShort)  +  0.9 * longRateShock(t, basepointsLong)


def flattener(t, basepointsShort, basepointsLong):
    if isinstance(t, (list, np.ndarray)):
        result = []
        for x in t:
            result.append(flattener(x, basepointsShort, basepointsLong))
        return result
    return 0.8 * shortRateShock(t, basepointsShort)  -  0.6 * longRateShock(t, basepointsLong)


def lowerBound(t):
    if isinstance(t, (list, np.ndarray)):
        result = []
        for x in t:
            result.append(lowerBound(x))
        return result
    return min(0, -100 + 5*t) # in basepoints


def getShockValue(t, shockScenario, parallel = 0, short = 0, long = 0):
    
    if shockScenario == '':
        return 0
    
    if shockScenario == 'Original':
        return 0
    
    
    
    if shockScenario == 'Parallel':
        return parallel
    
    if shockScenario == 'ParallelUp':
        return parallel
    
    if shockScenario == 'ParallelDown':
        return -parallel
    
    
    
    if shockScenario == 'Short':
        return shortRateShock(t, short)
    
    if shockScenario == 'ShortUp':
        return shortRateShock(t, short)
    
    if shockScenario == 'ShortDown':
        return shortRateShock(t, -short)
    
    
    
    if shockScenario == 'Long':
        return longRateShock(t, long)
    
    if shockScenario == 'LongUp':
        return longRateShock(t, long)
    
    if shockScenario == 'LongDown':
        return longRateShock(t, -long)
    
    
    
    if shockScenario == 'Flatten':
        return flattener(t, short, long)
    
    if shockScenario == 'Steepen':
        return steepener(t, short, long)
    
    
    raise Exception('I don\'t know a scenario of the name \'' + shockScenario + '\'')

In [ ]:
# define the parameters for the shock scenarios

shockParams = pd.DataFrame({'Currency': [], 'Parallel': [], 'Short': [], 'Long': []})
shockParams = shockParams.append({'Currency': 'EUR', 'Parallel': 200, 'Short': 250, 'Long': 100}, ignore_index = True)
shockParams = shockParams.append({'Currency': 'GBP', 'Parallel': 250, 'Short': 300, 'Long': 150}, ignore_index = True)
shockParams = shockParams.append({'Currency': 'USD', 'Parallel': 200, 'Short': 300, 'Long': 150}, ignore_index = True)

In [ ]:
#Define a function, which shifts a vector of interest rates according to a given shock scenario

def getShockedInterestRates(
        refdate,
        dates,
        interestRates,
        daycounter,
        shockScenario,
        parallel = 0,
        short = 0,
        long = 0,
        useLowerBound = True):
    if len(interestRates) != len(dates):
        raise RangeMismatch('You need to provide an equal number of interest rates and sampling dates.')
    
    shockedInterestRates = []
    
    for i in range(len(dates)):
        t = daycounter.yf(refdate, dates[i])
        rate = interestRates[i] + getShockValue(t, shockScenario, parallel, short, long)
        if useLowerBound:
            rate = max(rate, lowerBound(t))
        shockedInterestRates.append(rate)
    
    #print(shockedInterestRates)
    
    return shockedInterestRates

In [ ]:
# Define a function, which shifts a vector of discount factors according to a given shock scenario

def getShockedDiscountFactors(
        refdate,
        dates,
        interestRates,
        daycounter,
        shockScenario,
        parallel = 0,
        short = 0,
        long = 0,
        useLowerBound = True):
    
    if len(interestRates) != len(dates):
        raise RangeMismatch('You need to provide an equal number of discount factors and sampling dates.')
    
    shockedInteresRates = getShockedInterestRates(
        refdate,
        dates,
        interestRates,
        daycounter,
        shockScenario,
        parallel,
        short,
        long,
        useLowerBound
    )    
    
    shockedDiscountFactors = []
    #shockedDiscountFactors = analytics.vectorDouble(len(dates))
    #print(shockedDiscountFactors)
    
    for i in range(len(dates)):
        t = daycounter.yf(refdate, dates[i])
        rate = shockedInteresRates[i]/100/100 # are given in base points -> convert to decimal number
        shockedDiscountFactors.append(math.exp(-t*rate))
        #shockedDiscountFactors[i] = discountFactors[i] + getShockValue(t, shockScenario, parallel, short, long)
        #print(shockedDiscountFactors)
    
    #print("//////////////////////////////////////////////")
    #print(shockedInteresRates)
    #print("-----------------------")
    #print(shockedDiscountFactors)
    #print("//////////////////////////////////////////////")
    
    return shockedDiscountFactors

In [ ]:
# Define a function, which shifts a discount curve according to a given shock scenario   
# We now use these shifted discount factors to construct shifted discount curves     
    
def getShockedDiscountCurve(
        name,
        refdate,
        dates,
        interestRates,
        daycounterType,
        interpolationType,
        extrapolationType,
        shockScenario,
        parallel = 0,
        short = 0,
        long = 0,
        useLowerBound = True):
    
    shockedDFs = getShockedDiscountFactors(
        refdate,
        dates,
        interestRates,
        analytics.DayCounter(daycounterType),
        shockScenario,
        parallel,
        short,
        long,
        useLowerBound
    )    
    
    #print(shockedDFs)
    
    return analytics.DiscountCurve(
        name,
        refdate,
        dates,
        shockedDFs,
        daycounterType,
        interpolationType,
        extrapolationType
    )  

## The scenarios
### Plot the shock scenarios
In this section, we illustrate the interest rate scenarios by applying them to an interest rate curve with constant interest rate 0.

In [ ]:
refdate = dt.datetime(year = 2020, month = 1 , day = 1)


sampling_points_simple_example_yf = [1/365] # 1 day
sampling_points_simple_example_yf.extend( (np.arange(11)+1)/12 ) # 1 to 11 months
sampling_points_simple_example_yf.extend(np.arange(10)+1) # 1 to 10 years


sampling_points_simple_example = []
sampling_points_simple_example.append(refdate + dt.timedelta(days = 1))
for i in range(11):
    sampling_points_simple_example.append(refdate + dt.timedelta(days = (i+1)*30))
for i in range(10):
    year = refdate.year + i + 1
    month = refdate.month
    day = refdate.day
    sampling_points_simple_example.append(
        dt.datetime(year = year, month = month , day = day)
    )

In [ ]:
def plot_shock_scenarios(
    samplingPoints,
    shockParams
):
    fig = plt.figure(figsize=(16,8))
    ax = fig.gca()

    currency = 'EUR'
    parallel = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Parallel']
    short = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Short']
    long = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Long']

    #print(type(samplingPoints))

    #print(shortRateShock(short, samplingPoints))
    hline = ax.axhline(y=0, xmin=0, xmax=1, linewidth=1, zorder = 30)
    ax.plot(samplingPoints, [0]*len(samplingPoints), '.-', label = 'Original', zorder = 20)
    ax.plot(samplingPoints, [parallel]*len(samplingPoints), '.-', label = 'ParallelUp', zorder = 20)
    ax.plot(samplingPoints, [-parallel]*len(samplingPoints), '.-', label = 'ParallelDown', zorder = 20)
    ax.plot(samplingPoints, shortRateShock(samplingPoints, short), '.-', label = 'ShortUp', zorder = 20)
    ax.plot(samplingPoints, shortRateShock(samplingPoints, -short), '.-', label = 'ShortDown', zorder = 20)
    ax.plot(samplingPoints, longRateShock(samplingPoints, long), '.-', label = 'LongUp', zorder = 20)
    ax.plot(samplingPoints, longRateShock(samplingPoints, -long), '.-', label = 'LongDown', zorder = 20)
    ax.plot(samplingPoints, flattener(samplingPoints, short, long), '.-', label = 'Flattener', zorder = 20)
    ax.plot(samplingPoints, steepener(samplingPoints, short, long), '.-', label = 'Steepener', zorder = 20)


    plt.grid(alpha=grid_alpha) 
    plt.xlabel('Expiry (in years)')
    plt.ylabel('Interest rate (in base points)')
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.legend(loc='lower right').set_zorder(100)

    plt.show()

plot_shock_scenarios(sampling_points_simple_example_yf, shockParams)

In [ ]:
def plot_shocked_interest_rates(
    refdate,
    dates,
    interestRates,
    daycounter,
    useLowerBound = True
):
    fig = plt.figure(figsize=(16,8))
    ax = fig.gca()

    currency = 'EUR'
    parallel = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Parallel']
    short = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Short']
    long = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Long']

    #print(type(samplingPoints))

    #print(shortRateShock(short, samplingPoints))
    hline = ax.axhline(y=0, xmin=0, xmax=1, linewidth=1, zorder = 30) # constant 0
    
    shockScenarios = ['Original', 'ParallelUp', 'ParallelDown', 'ShortUp', 'ShortDown', 'LongUp', 'LongDown', 'Flatten', 'Steepen']
    for shockScenario in shockScenarios:
        ir = getShockedInterestRates(
            refdate,
            dates,
            interestRates,
            daycounter,
            shockScenario,
            parallel,
            short,
            long,
            useLowerBound
        )
        year_fractions = []
        for i in range(len(dates)):
            year_fractions.append(daycounter.yf(refdate, dates[i]))
        
        ax.plot(year_fractions, ir, '.-', label = shockScenario, zorder = 20)

    
    plt.grid(alpha=grid_alpha)    
    plt.xlabel('Expiry (in years)')
    plt.ylabel('Interest rate (in base points)')
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.legend(loc='lower right').set_zorder(100)

    plt.show()


interestRates = [0] * len(sampling_points_simple_example)

plot_shocked_interest_rates(
    refdate,
    sampling_points_simple_example,
    interestRates,
    analytics.DayCounter(daycounter_type_standard),
    False
)


plot_shocked_interest_rates(
    refdate,
    sampling_points_simple_example,
    interestRates,
    analytics.DayCounter(daycounter_type_standard),
    True
)

### Plot the shock scenarios (discount factors)

In [ ]:
def plot_shocked_discount_factors(
    refdate,
    dates,
    interestRates,
    daycounter,
    useLowerBound = True
):
    fig = plt.figure(figsize=(16,8))
    ax = fig.gca()

    currency = 'EUR'
    parallel = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Parallel']
    short = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Short']
    long = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Long']

    #print(type(samplingPoints))

    #print(shortRateShock(short, samplingPoints))
    hline = ax.axhline(y=1, xmin=0, xmax=1, linewidth=1, zorder = 30)
    shockScenarios = ['Original', 'ParallelUp', 'ParallelDown', 'ShortUp', 'ShortDown', 'LongUp', 'LongDown', 'Flatten', 'Steepen']
    for shockScenario in shockScenarios:
        df = getShockedDiscountFactors(
            refdate,
            dates,
            interestRates,
            daycounter,
            shockScenario,
            parallel,
            short,
            long,
            useLowerBound
        )
        year_fractions = []
        for i in range(len(dates)):
            year_fractions.append(daycounter.yf(refdate, dates[i]))
        
        ax.plot(year_fractions, df, '.-', label = shockScenario, zorder = 20)

    
    plt.grid(alpha=grid_alpha) 
    plt.xlabel('Expiry (in years)')
    plt.ylabel('Discount factor')
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.legend(loc='lower right').set_zorder(100)

    plt.show()


interestRates = [0] * len(sampling_points_simple_example)

plot_shocked_discount_factors(
    refdate,
    sampling_points_simple_example,
    interestRates,
    analytics.DayCounter(daycounter_type_standard),
    False
)

plot_shocked_discount_factors(
    refdate,
    sampling_points_simple_example,
    interestRates,
    analytics.DayCounter(daycounter_type_standard),
    True
)

#### Plot them again, using discount curves

In [ ]:
def plot_shocked_discount_curves(
    name,
    refdate,
    dates,
    discountFactors,
    daycounterType,
    interpolationType,
    extrapolationType,
    shockParams,
    useLowerBound = True
):
    fig = plt.figure(figsize=(16,8))
    ax = fig.gca()

    currency = 'EUR'
    parallel = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Parallel']
    short = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Short']
    long = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Long']
    
    daycounter = analytics.DayCounter(daycounterType)

    #print(type(samplingPoints))

    #print(shortRateShock(short, samplingPoints))
    hline = ax.axhline(y=1, xmin=0, xmax=1, linewidth=1, zorder = 30)
    shockScenarios = ['Original', 'ParallelUp', 'ParallelDown', 'ShortUp', 'ShortDown', 'LongUp', 'LongDown', 'Flatten', 'Steepen']
    for shockScenario in shockScenarios:
        dc = getShockedDiscountCurve(
            name + '_' + shockScenario,
            refdate,
            dates,
            discountFactors,
            daycounterType,
            interpolationType,
            extrapolationType,
            shockScenario,
            parallel,
            short,
            long,
            useLowerBound
        )
        
        year_fractions = []
        for i in range(len(dates)):
            year_fractions.append(daycounter.yf(refdate, dates[i]))
        values = analytics.vectorDouble()
        dc.value(values, refdate, dates)
            
        ax.plot(year_fractions, values, '.-', label = shockScenario, zorder = 20)

        
    plt.grid(alpha=grid_alpha) 
    plt.xlabel('Expiry (in years)')
    plt.ylabel('Discount factor')
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.legend(loc='lower right').set_zorder(100)

    plt.show()
    
    
plot_shocked_discount_curves(
    name = 'dc_linear',
    refdate = refdate,
    dates = sampling_points_simple_example,
    discountFactors = [0.0] * len(sampling_points_simple_example),
    daycounterType = daycounter_type_standard,
    interpolationType = interpolation_type_standard,
    extrapolationType = extrapolation_type_standard,
    shockParams = shockParams,
    useLowerBound = False
)    
    
plot_shocked_discount_curves(
    name = 'dc_linear',
    refdate = refdate,
    dates = sampling_points_simple_example,
    discountFactors = [0.0] * len(sampling_points_simple_example),
    daycounterType = daycounter_type_standard,
    interpolationType = interpolation_type_standard,
    extrapolationType = extrapolation_type_standard,
    shockParams = shockParams,
    useLowerBound = True
)

## Example
In this section, we apply the interest rate scenarios to a more realistic interest rate curve. The curve we use is taken from *Rundschreiben 06/2019 (BA) - Zinsänderungsrisiken im Anlagenbuch* an official regulatory document issued by BaFin.

In [ ]:
# Define example data

example_BaFin_interest_rate_basepoints = [
    -57,
    -60,
    -62,
    -64,
    -69,
    -68,
    -64,
    -56,
    -48,
    -39,
    -30,
    -21,
    -13,
    -6,
    1,
    7,
    13,
    18,
    22,
    26,
    30,
    33,
    36,
    38,
    41,
    43,
    45,
    47,
    49,
    50,
    52,
    53,
    55
]

example_BaFin_sampling_points = [
    refdate + dt.timedelta(days = round(yf*365)) for yf in [0.25, 0.5, 0.75]
]

for i in range(30):
    year = refdate.year + i + 1
    month = refdate.month
    day = refdate.day
    example_BaFin_sampling_points.append(
        dt.datetime(year = year, month = month , day = day)
    )

### Plot all scenarios

In [ ]:
plot_shocked_interest_rates(
    refdate,
    example_BaFin_sampling_points,
    example_BaFin_interest_rate_basepoints,
    analytics.DayCounter(daycounter_type_standard),
    False
)

plot_shocked_interest_rates(
    refdate,
    example_BaFin_sampling_points,
    example_BaFin_interest_rate_basepoints,
    analytics.DayCounter(daycounter_type_standard),
    True
)

### Plot individual scenarios

In [ ]:
# Define a function, which applies and then plots a single scenario

def plot_shocked_interest_rates_single(
    refdate,
    dates,
    interestRates,
    daycounter,
    shockScenario,
    useLowerBound = True,
    plotOriginal = True,
    plotLowerBound = True
):
    fig = plt.figure(figsize=(16,8))
    ax = fig.gca()

    currency = 'EUR'
    parallel = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Parallel']
    short = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Short']
    long = shockParams.loc[shockParams['Currency'] == currency].loc[0]['Long']
    
    
    year_fractions = []
    for i in range(len(dates)):
        year_fractions.append(daycounter.yf(refdate, dates[i]))
        
        
    if plotOriginal:
        ax.plot(year_fractions, interestRates, '.-', label = 'Original', zorder = 20)

        
    # plot chosen scenario  
    if shockScenario != '' and shockScenario != 'Original':
        ir = getShockedInterestRates(
            refdate,
            dates,
            interestRates,
            daycounter,
            shockScenario,
            parallel,
            short,
            long,
            useLowerBound
        )

        ax.plot(year_fractions, ir, '.-', label = shockScenario, zorder = 20)
    
    
    if plotLowerBound:
        ax.plot(year_fractions, lowerBound(year_fractions), '--', label = 'Lower Bound', zorder = 10)

    
    plt.grid(alpha=grid_alpha)    
    plt.xlabel('Expiry (in years)')
    plt.ylabel('Interest rate (in base points)')
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.legend(loc='lower right').set_zorder(100)

    plt.show()

In [ ]:
# Define the function, which we're going to use in combination with widgets in order to plot
# the scenario specified by the user.

def showPlot(
    shockScenario = 'Original',
    useLowerBound = True
):
    plot_shocked_interest_rates_single(
        refdate = refdate,
        dates = example_BaFin_sampling_points,
        interestRates = example_BaFin_interest_rate_basepoints,
        daycounter = analytics.DayCounter(daycounter_type_standard),
        shockScenario = shockScenario,
        useLowerBound = useLowerBound,
        plotOriginal = True,
        plotLowerBound = useLowerBound
    )

In [ ]:
# Use widgets to make the plot interactive

combobox = widgets.Combobox(
    # value='Flatten',
    placeholder='Choose Scenario',
    options=['ParallelUp', 'ParallelDown', 'ShortUp', 'ShortDown', 'LongUp', 'LongDown', 'Flatten', 'Steepen'],
    description='Scenario:',
    ensure_option=True,
    disabled=False
)

checkbox = widgets.Checkbox(
    value=False,
    description='Use lower bound',
    disabled=False,
    indent=True
)

widgets.interact(
    showPlot,
    shockScenario = combobox,
    useLowerBound = checkbox
)

# TODO

- XXXX